In [265]:
import requests
import json
import pandas as pd

## Obtain API Key to Access Data

In [266]:
api_key = 'ZXCH94jmLisJ8b72hhJC31sCKyZHmEwBxTcp763u'

## Obtain Data

In [267]:
link = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?&_fields=school.name,id,2015.cost.avg_net_price.overall,2015.cost.net_price.public.by_income_level.0-30000,2015.cost.net_price.public.by_income_level.110001-plus,2015.cost.net_price.public.by_income_level.30001-48000,2015.cost.net_price.public.by_income_level.48001-75000,2015.cost.net_price.public.by_income_level.75001-110000,2015.cost.net_price.public.by_income_level.75000-plus,2015.cost.net_price.private.by_income_level.0-30000,2015.cost.net_price.private.by_income_level.110001-plus,2015.cost.net_price.private.by_income_level.30001-48000,2015.cost.net_price.private.by_income_level.48001-75000,2015.cost.net_price.private.by_income_level.75001-110000,2015.cost.net_price.private.by_income_level.75000-plus,2015.admissions.sat_scores.average.overall,2015.admissions.sat_scores.25th_percentile.critical_reading,2015.admissions.sat_scores.25th_percentile.math,2015.admissions.sat_scores.25th_percentile.writing,2015.admissions.sat_scores.75th_percentile.critical_reading,2015.admissions.sat_scores.75th_percentile.math,2015.admissions.sat_scores.75th_percentile.writing,2015.admissions.act_scores.25th_percentile.cumulative,2015.admissions.act_scores.75th_percentile.cumulative,2015.admissions.admission_rate.overall,2015.student.size,2015.cost.attendance.academic_year,school.tuition_revenue_per_fte,school.instructional_expenditure_per_fte,school.degrees_awarded.predominant,2015.aid.pell_grant_rate,2015.aid.federal_loan_rate,2015.aid.median_debt.completers.overall,2013.repayment.1_yr_repayment.completers,2013.earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings&school.main_campus=1&school.operating=1&school.online_only=0&school.institutional_characteristics.level=1&2015.academics.program_available.assoc_or_bachelors=True'

In [268]:
params = {
        'api_key':api_key,
        '_page':1,
        '_per_page':100}

In [269]:
res = requests.get(link, params)
schools = json.loads(res.content)

In [270]:
school_df = pd.DataFrame(schools['results'])

### Code below gives error message only when it's done collecting data on all schools matching criteria.  

In [299]:
for x in range (2, 202):
    params = {
        'api_key':api_key,
        '_page':x,
        '_per_page':100}
    res = requests.get(link, params)
    sc = json.loads(res.content)
    sc2 = pd.DataFrame(sc['results'])
    school_df = school_df.append(sc2)

In [272]:
school_df.shape

(2009, 35)

In [273]:
school_df.isnull().sum()

2013.earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings     174
2013.repayment.1_yr_repayment.completers                                165
2015.admissions.act_scores.25th_percentile.cumulative                   863
2015.admissions.act_scores.75th_percentile.cumulative                   863
2015.admissions.admission_rate.overall                                  428
2015.admissions.sat_scores.25th_percentile.critical_reading             906
2015.admissions.sat_scores.25th_percentile.math                         897
2015.admissions.sat_scores.25th_percentile.writing                     1337
2015.admissions.sat_scores.75th_percentile.critical_reading             906
2015.admissions.sat_scores.75th_percentile.math                         897
2015.admissions.sat_scores.75th_percentile.writing                     1337
2015.admissions.sat_scores.average.overall                              827
2015.aid.federal_loan_rate                                                5
2015.aid.med

# Data Cleaning

#### Feature Engineer: Filter Minimum Net Price, Maximum Net Price and Average Net Price

In [274]:
school_df['High_Net_Price'] = school_df[['2015.cost.avg_net_price.overall',
       '2015.cost.net_price.private.by_income_level.0-30000',
       '2015.cost.net_price.private.by_income_level.110001-plus',
       '2015.cost.net_price.private.by_income_level.30001-48000',
       '2015.cost.net_price.private.by_income_level.48001-75000',
       '2015.cost.net_price.private.by_income_level.75000-plus',
       '2015.cost.net_price.private.by_income_level.75001-110000',
       '2015.cost.net_price.public.by_income_level.0-30000',
       '2015.cost.net_price.public.by_income_level.110001-plus',
       '2015.cost.net_price.public.by_income_level.30001-48000',
       '2015.cost.net_price.public.by_income_level.48001-75000',
       '2015.cost.net_price.public.by_income_level.75000-plus',
       '2015.cost.net_price.public.by_income_level.75001-110000']].max(axis=1)

In [275]:
school_df['Low_Net_Price'] = school_df[['2015.cost.avg_net_price.overall',
       '2015.cost.net_price.private.by_income_level.0-30000',
       '2015.cost.net_price.private.by_income_level.110001-plus',
       '2015.cost.net_price.private.by_income_level.30001-48000',
       '2015.cost.net_price.private.by_income_level.48001-75000',
       '2015.cost.net_price.private.by_income_level.75000-plus',
       '2015.cost.net_price.private.by_income_level.75001-110000',
       '2015.cost.net_price.public.by_income_level.0-30000',
       '2015.cost.net_price.public.by_income_level.110001-plus',
       '2015.cost.net_price.public.by_income_level.30001-48000',
       '2015.cost.net_price.public.by_income_level.48001-75000',
       '2015.cost.net_price.public.by_income_level.75000-plus',
       '2015.cost.net_price.public.by_income_level.75001-110000']].min(axis=1)

In [276]:
school_df['Avg_Net_Price'] = school_df['2015.cost.avg_net_price.overall']

In [277]:
school_df['Total_Cost'] =  school_df['2015.cost.avg_net_price.overall']


#### Feature Engineering:  Combine SAT Scores to Total (Eliminate Writing Score as many schools do not report) 

In [278]:
school_df['SAT_25th_Cum'] = school_df['2015.admissions.sat_scores.25th_percentile.critical_reading'] + school_df['2015.admissions.sat_scores.25th_percentile.math']

In [279]:
school_df['SAT_75th_Cum'] = school_df['2015.admissions.sat_scores.75th_percentile.critical_reading'] + school_df['2015.admissions.sat_scores.75th_percentile.math']

#### Feature Engineering:   ACT Scores

In [280]:
school_df['ACT_25th'] = school_df['2015.admissions.act_scores.25th_percentile.cumulative']

In [281]:
school_df['ACT_75th'] = school_df['2015.admissions.act_scores.75th_percentile.cumulative']

#### Filter out columns that have been combined in feature engineering

In [282]:
school_2 = school_df[['2013.earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings',
       '2015.admissions.admission_rate.overall',
       '2015.aid.federal_loan_rate', '2015.aid.median_debt.completers.overall',
       '2015.aid.pell_grant_rate',
       '2015.student.size', 'id', 'school.degrees_awarded.predominant', 'school.name', 'High_Net_Price', 'Low_Net_Price',
       'Avg_Net_Price', 'SAT_25th_Cum', 'SAT_75th_Cum', 'ACT_25th',
       'ACT_75th', 'Total_Cost']]

In [283]:
school_2.isnull().sum()

2013.earnings.10_yrs_after_entry.working_not_enrolled.mean_earnings    174
2015.admissions.admission_rate.overall                                 428
2015.aid.federal_loan_rate                                               5
2015.aid.median_debt.completers.overall                                162
2015.aid.pell_grant_rate                                                 5
2015.student.size                                                        3
id                                                                       0
school.degrees_awarded.predominant                                       0
school.name                                                              0
High_Net_Price                                                         106
Low_Net_Price                                                          106
Avg_Net_Price                                                          106
SAT_25th_Cum                                                           906
SAT_75th_Cum             

In [284]:
school_2.columns = ['Earnings_10yrs',
       'Admission_Rate', 'Fed_Loan_Rate',
       'Median_Debt', 'Pell_Grant_Rate','Student_Size', 'ID',
       'Degrees_Awarded', 'School_Name','High_Net_Price', 'Low_Net_Price',
       'Avg_Net_Price', 'SAT_25th_Cum', 'SAT_75th_Cum', 'ACT_25th',
       'ACT_75th', 'Total_Cost']

## Drop schools out of dataset without cost & earnings information

Although we may lose some important schools, the unavailable information makes it worthless in this study.  Admission Rate & Median Debt are less important so we will keep and zero out null values for these columns.

In [285]:
school_2 = school_2.dropna(subset=['Avg_Net_Price','Total_Cost'], how='all')

In [286]:
school_2 = school_2.dropna(subset=['Earnings_10yrs'], how='all')

In [287]:
school_2.isnull().sum()

Earnings_10yrs       0
Admission_Rate     295
Fed_Loan_Rate        0
Median_Debt         49
Pell_Grant_Rate      0
Student_Size         0
ID                   0
Degrees_Awarded      0
School_Name          0
High_Net_Price       0
Low_Net_Price        0
Avg_Net_Price        0
SAT_25th_Cum       689
SAT_75th_Cum       689
ACT_25th           646
ACT_75th           646
Total_Cost           0
dtype: int64

In [288]:
school_2.shape

(1773, 17)

Fill missing admission rates with mean  

In [289]:
school_2['Admission_Rate']= school_2['Admission_Rate'].fillna((school_2['Admission_Rate'].mean()))

Fill missing median debt with mean

In [290]:
school_2['Median_Debt'] = school_2['Median_Debt'].fillna((school_2['Median_Debt'].mean()))

In [291]:
school_2.isnull().sum()

Earnings_10yrs       0
Admission_Rate       0
Fed_Loan_Rate        0
Median_Debt          0
Pell_Grant_Rate      0
Student_Size         0
ID                   0
Degrees_Awarded      0
School_Name          0
High_Net_Price       0
Low_Net_Price        0
Avg_Net_Price        0
SAT_25th_Cum       689
SAT_75th_Cum       689
ACT_25th           646
ACT_75th           646
Total_Cost           0
dtype: int64

## Dateframe without SAT/ ACT Scores

In [292]:
school_no_scores = school_2[['Earnings_10yrs', 'Admission_Rate', 'Fed_Loan_Rate', 'Median_Debt',
       'Pell_Grant_Rate', 'Student_Size', 'ID', 'Degrees_Awarded',
       'School_Name', 'High_Net_Price', 'Low_Net_Price', 'Avg_Net_Price', 'Total_Cost']]

In [293]:
school_no_scores.to_csv(path_or_buf = 'School_No_Scores', index=False)

## Dateframe with SAT/ ACT Scores

In [295]:
school_scores = school_scores.dropna()

In [296]:
school_scores.isnull().sum()

Earnings_10yrs     0
Admission_Rate     0
Fed_Loan_Rate      0
Median_Debt        0
Pell_Grant_Rate    0
Student_Size       0
ID                 0
Degrees_Awarded    0
School_Name        0
High_Net_Price     0
Low_Net_Price      0
Avg_Net_Price      0
SAT_25th_Cum       0
SAT_75th_Cum       0
ACT_25th           0
ACT_75th           0
Total_Cost         0
dtype: int64

In [297]:
school_scores.shape

(1052, 17)

In [298]:
school_scores.to_csv(path_or_buf = 'School_Scores', index=False)